In [1]:
# %%capture

# !pip install pyspellchecker==0.7.0
# !pip install -q transformers

# !pip install sentencepiece
# !pip install git+https://github.com/csebuetnlp/normalizer

# ! pip install bangla==0.0.2
# # !pip install num2words
!nvidia-smi

Tue Nov  1 17:40:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:65:00.0 Off |                  Off |
| 30%   39C    P8    29W / 300W |   3551MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
# from num2words import num2words
import os
from tqdm.auto import tqdm
tqdm.pandas()

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import transformers

import torch
import bangla
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

from spellchecker import SpellChecker

import warnings
warnings.filterwarnings("ignore")

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True,nb_workers=8)

os.environ["TOKENIZERS_PARALLELISM"] = "true"

print(torch.__version__)
spell = SpellChecker()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/ansary/anaconda3/envs/mobassir/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.12.1


In [3]:
!pwd

/home/ansary/Shabab/tafsir_ibn_kathir


In [4]:
tafsir_en = pd.read_csv('/home/ansary/Shabab/tafsir_ibn_kathir/tafsir_ibn_kathir.csv')
tafsir_en.head()

,Surah,Ayah,Tafsir
0,1,1,Introduction to Fatihah\nWhich was revealed in...
1,1,2,1\nبِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ\n1....
2,1,3,2\nٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ\n2. ...
3,1,4,3\nٱلرَّحْمَـٰنِ ٱلرَّحِيمِ\n3. Ar-Rahman (the...
4,1,5,4\nمَـٰلِكِ يَوْمِ ٱلدِّينِ\n4. The Owner of t...


In [5]:
%%time

def tafsir_cleaner(tafsir):
    res = re.sub('\'','',tafsir)
    res = re.sub('\n','.',res)
    
    res = re.sub('﴾','',res)
    res = re.sub('﴿','',res)
    
    res = re.sub('«','',res)
    res = re.sub('»','',res)
    
    res = res.replace('Prev.Next','')
    return res
    
tafsir_en["Tafsir"]=tafsir_en.Tafsir.progress_apply(lambda tafsir: tafsir_cleaner(tafsir))

100%|████████████████████████████████████| 2029/2029 [00:00<00:00, 12627.45it/s]

CPU times: user 150 ms, sys: 18.2 ms, total: 168 ms
Wall time: 166 ms


In [6]:
def tag_arabic_text(text,ar_pattern=u'[\u0600-\u06FF]+',english_only = False):
    # remove multiple spaces
    data=re.sub(' +', ' ',text)
    texts=[]
    if "।" in data:punct="।"
    elif "." in data:punct="."
    else:punct="\n"
    for text in data.split(punct):    
        # create start and end
        text="start"+text+"end"
        # tag text
        parts=re.split(ar_pattern, text)
        parts=[p for p in parts if p.strip()]
        parts=set(parts)
        for m in parts:
            if len(m.strip())>1:text=text.replace(m,f"</ar>{m}<ar>")
        # clean-up invalid combos
        text=text.replace("</ar>start",'')
        text=text.replace("end<ar>",'')
        texts.append(text)
    text=f"{punct}".join(texts)
    if(english_only):
        #https://stackoverflow.com/questions/55656429/replace-or-remove-html-tag-content-python-regex
        return re.sub(r'(?s)<ar>.*?</ar>', '', text)
    return text

In [7]:

tafsir_en["Tafsir_en"]=tafsir_en.Tafsir.progress_apply(lambda tafsir_eng: tag_arabic_text(tafsir_eng,english_only=True))

100%|█████████████████████████████████████| 2029/2029 [00:01<00:00, 1706.15it/s]


In [8]:
#tafsir_en["Tafsir_en"][0]

In [9]:
#https://stackoverflow.com/questions/71620622/is-there-a-way-to-identify-and-create-a-list-of-all-acronyms-in-a-dataframe

# tafsir_en["detected_misspelled"] = tafsir_en.assign(
#     misspelled=tafsir_en["Tafsir_en"].str.split().map(spell.unknown),
#     acronyms=tafsir_en["Tafsir_en"].str.findall(r"([A-Z]{2,})").map(set)
# )[["misspelled", "acronyms"]].apply(lambda row: set.union(*row), axis=1)

def find_unknown_words(text):
    list_of_words = text.split()
    list_of_words = [re.sub(r'[^A-Za-z0-9-]+', ' ', x) for x in list_of_words]
    
    word_list = []
    for i in range(len(list_of_words)):
        temp = list_of_words[i].split()
        if(len(temp) == 1):
            word_list.append(temp[0])
        else:
            for j in range(len(temp)):
                word_list.append(temp[j])
            
    return ' '.join(list(spell.unknown(word_list)))


In [10]:
tafsir_en["detected_unknowns"] = tafsir_en.Tafsir_en.progress_apply(lambda tafsir_eng: find_unknown_words(tafsir_eng))

100%|██████████████████████████████████████| 2029/2029 [00:03<00:00, 572.31it/s]


In [11]:
tafsir_en.head()

,Surah,Ayah,Tafsir,Tafsir_en,detected_unknowns
0,1,1,Introduction to Fatihah.Which was revealed in ...,Introduction to Fatihah.Which was revealed in ...,furqan surahs atas jibril hasan ruqyah jabir a...
1,1,2,1.بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ.1. I...,"1..1. In the Name of Allah, the Most Gracious,...",ash-shafi al-baraah nadim hasan al-hasan muqar...
2,1,3,2.ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ.2. Al...,"2..2. Al-Hamd be to Allah, the Lord of all tha...",ar-rabb 23-24 hamd alam lillah hasan dhikr ana...
3,1,4,3.ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ.3. Ar-Rahman (the M...,"3..3. Ar-Rahman (the Most Gracious), Ar-Rahim ...",al-qurtubi sahih ar-rahim ar-rahman 49-50 rabb...
4,1,5,4.مَـٰلِكِ يَوْمِ ٱلدِّينِ.4. The Owner of the...,4..4. The Owner of the Day of Recompense.. Ind...,al-malik salaf umar allahs ad-din ilaha huwa q...


In [12]:
unknown_words = list(tafsir_en['detected_unknowns'].str.split(' ', expand=True).stack().unique())
len(unknown_words)

5856

In [13]:
count_unknown = tafsir_en['detected_unknowns'].str.split(' ', expand=True).stack().value_counts()

In [14]:
count_unknown

allahs     1158
mujahid     751
qatadah     740
hadith      665
ayat        611
           ... 
udgment       1
unj           1
remely        1
iful          1
rijalun       1
Length: 5856, dtype: int64

In [15]:


# en_text = "all praise to him."

# model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
# tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# # translate en to bn
# tokenizer.src_lang = "en"
# encoded_en = tokenizer(en_text, return_tensors="pt")
# generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("bn"))
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


from code above, it seems like the performance of m2m100_418M is not good enough, nllb should be much better than m2m100_418M.

In [16]:

print(torch.__version__)
transformers.__version__

1.12.1


'4.22.1'

# single sample inference test

thanks to [banglanmt](https://huggingface.co/csebuetnlp/banglat5_nmt_en_bn/discussions)

In [17]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_en_bn").to(torch_device)
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_en_bn",use_fast=True)

def translate_en_bn(input_sentence):
    input_ids = tokenizer(normalize(input_sentence), return_tensors="pt").input_ids
    input_ids = input_ids.to(torch_device)
    generated_tokens = model.generate(input_ids)#max_length=1536
    decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]
    decoded_tokens=decoded_tokens.replace("<pad>","").replace("</s>","")
    sen=decoded_tokens.split()
    words=[w for w in sen if w.strip()]
    sen="".join(words)
    return decoded_tokens

text=translate_en_bn("alhamdulillah for everything.")
print(text)

cuda
 সবকিছুর জন্য আলহামদুলিল্লাহ।


In [18]:
#tag_arabic_text(tafsir_en.Tafsir[0],english_only=False)


In [19]:
%%time

en_text = '''In the Name of Allah, the Most Gracious, the Most Merciful. <ar> ﴿الْحَمْدُ للَّهِ رَبِّ الْعَـلَمِينَ﴾ </ar>
(Allah, the Exalted, said, `I have divided the prayer (Al-Fatihah) into two halves between 
Myself and My servant, and My servant shall have what he asks for.
If he says,' <ar> بِسْمِ اللَّهِ الرَّحْمَـنِ الرَّحِيمِ </ar> Allah says, `My servant has praised Me.' '''

def EN_AR_to_BN_AR_Translator(en_text,tag_text = False):
    '''
    translates multilingual english-arabic code mixed text into 
    multilingual bengali-arabic code mixed text
    ''' 
    if(tag_text):
        en_text = tag_arabic_text(en_text,english_only=False)
    
    sentenceEnders = re.compile('[.!?]')
    sentences = sentenceEnders.split(en_text)
    main_list = []
    for i in range(len(sentences)):
        
        list_str = sentences[i].split('<ar>')
        if(len(list_str) == 1):
            main_list.append(list_str[0])
        else:
            for j in range(len(list_str)):
                if('</ar>' in list_str[j]):
                    list_str1 = list_str[j].split('</ar>')
                    main_list.append("<ar>"+list_str1[0]+"</ar>")
                    main_list.append(list_str1[1])
                else:
                    main_list.append(list_str[j])

    while(" " in main_list):
        main_list.remove(" ")
    for idx in range(len(main_list)):
        if('<ar>' not in main_list[idx] or '</ar>' not in main_list[idx]):
            
            output_sentence = []
            for word in main_list[idx].split():
                output_sentence.append(word)
     
            main_list[idx] = ' '.join(output_sentence)
            #numerizer
            main_list[idx] = bangla.convert_english_digit_to_bangla_digit(main_list[idx])
            # multilingual english-arabic to multilingual bengali-arabic
            try:
                if len(main_list[idx])>1:
                    main_list[idx]=translate_en_bn(main_list[idx])
                            
            except:
                print("failed -> ",main_list[idx])
    
    bn_mlt = " ".join(main_list)
    bn_mlt = re.sub(' ্ ','',bn_mlt)
    bn_mlt = re.sub("\\'","",bn_mlt)#replace \'
    bn_mlt = re.sub('<unk>','',bn_mlt)
    return bn_mlt
        
EN_AR_to_BN_AR_Translator(en_text,tag_text = False)

CPU times: user 1.5 s, sys: 16.1 ms, total: 1.52 s
Wall time: 1.52 s


' আল্লাহ তাআলার নামে, যিনি পরম দয়ালু, পরম দয়ালু <ar> ﴿الْحَمْدُ للَّهِ رَبِّ الْعَـلَمِينَ﴾ </ar>  (আল্লাহ, যিনি মহান, বললেন, আমি আমার ও আমার বান্দার মধ্যে নামাযকে (আল-ফাতিহা) দুই ভাগে বিভক্ত করে দিয়েছি। আমার বান্দা যা চাইবে, তা-ই পাবে।  যদি সে বলে, <ar> بِسْمِ اللَّهِ الرَّحْمَـنِ الرَّحِيمِ </ar>  আল্লাহ বলেন, আমার বান্দা আমার প্রশংসা করেছে '

quality of translation looks much better!

In [20]:
%%time
tafsir_en["Tafsir_bn"]=tafsir_en.Tafsir.progress_apply(lambda tafsir_eng: EN_AR_to_BN_AR_Translator(tafsir_eng,tag_text = True))

#tafsir_en["Tafsir_bn"]=tafsir_en.Tafsir.parallel_apply(lambda tafsir_eng: EN_AR_to_BN_AR_Translator(tafsir_eng,tag_text = True))

# tafsir_en["Tafsir_bn"] = tafsir_en["Tafsir"]
# for i in range(5):
#     tafsir_en["Tafsir_bn"][i] = EN_AR_to_BN_AR_Translator(tafsir_en.Tafsir[i],tag_text = True)


100%|████████████████████████████████████| 2029/2029 [10:17:48<00:00, 18.27s/it]

CPU times: user 10h 17min 30s, sys: 9.62 s, total: 10h 17min 39s
Wall time: 10h 17min 48s


In [21]:
tafsir_en["Tafsir_bn"][0]

' ফাতিমার পরিচিতি  যা মক্কায় অবতীর্ণ হয়েছিল  আল-ফাতিহা এবং এর বিভিন্ন নামের অর্থ  এই সুরাকে বলা হয়  - আল-ফাতিহা, অর্থাৎ, কিতাবের প্রবর্তক, সূরা যার মাধ্যমে প্রার্থনা শুরু হয়  - বেশিরভাগ পণ্ডিতদের মতে এটিকে উম্মে আল-কিতাব (বুকের মা) নামেও ডাকা হয়।   আত-তিরমিদি কর্তৃক লিখিত একটি নির্ভরযোগ্য হাদিসে আবু হুরায়রাহ বলেন যে, আল্লাহর রসূল বলেছেন,  <ar>الْحَمْدُ للهِ رَبَ الْعَالَمِينَ أُمُّ الْقُرْآنِ وَأُمُّ الْكِتَابِ وَالسَّبْعُ الْمَثَانِي وَالْقُرْآنُ الْعَظِيمُ</ar>   আল-হামদু লিল্লাহি রব্বিল-আলামিন কুরআনের মা, কিতাবের মা এবং পবিত্র কুরআনের সাত বার আয়াত  একে আল-হামদ ও আস-সালাহও বলা হয়, কারণ নবী বলেছেন, তাঁর পালনকর্তা বলেছেন,  <ar>قَسَمْتُ الصَّلَاةَ بَيْنِي وَبَيْنَ عَبْدِي نِصْفَيْنِ، فَإِذَا قَالَ الْعَبْدُ:الْحَمْدُ للهِ رَبِّ الْعَالَمِنَ، قَالَ اللهُ: حَمِدَنِي عَبْدِي</ar>   প্রার্থনা (১ e , আল-ফাতিহা) আমার ও আমার বান্দাদের মধ্যে দুই ভাগে বিভক্ত  যখন বান্দা বলে, সকল প্রশংসা আল্লাহর, যিনি জীবিত, আল্লাহ বলেন, আমার বান্দা আমার প্রশংসা করেছে।  আল-ফাতিহাকে সালাহ বলা হত, কারণ এটি 

In [22]:

tafsir_en.to_csv('/home/ansary/Shabab/tafsir_ibn_kathir/Multilingual_bn_ar_tafsir_ibn_kathir.csv',index = False)

In [23]:
tafsir_en.head()

,Surah,Ayah,Tafsir,Tafsir_en,detected_unknowns,Tafsir_bn
0,1,1,Introduction to Fatihah.Which was revealed in ...,Introduction to Fatihah.Which was revealed in ...,furqan surahs atas jibril hasan ruqyah jabir a...,ফাতিমার পরিচিতি যা মক্কায় অবতীর্ণ হয়েছিল ...
1,1,2,1.بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ.1. I...,"1..1. In the Name of Allah, the Most Gracious,...",ash-shafi al-baraah nadim hasan al-hasan muqar...,১ <ar>بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ...
2,1,3,2.ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ.2. Al...,"2..2. Al-Hamd be to Allah, the Lord of all tha...",ar-rabb 23-24 hamd alam lillah hasan dhikr ana...,২ <ar>ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ<...
3,1,4,3.ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ.3. Ar-Rahman (the M...,"3..3. Ar-Rahman (the Most Gracious), Ar-Rahim ...",al-qurtubi sahih ar-rahim ar-rahman 49-50 rabb...,৩ <ar>ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ</ar> ৩ আর-রহ...
4,1,5,4.مَـٰلِكِ يَوْمِ ٱلدِّينِ.4. The Owner of the...,4..4. The Owner of the Day of Recompense.. Ind...,al-malik salaf umar allahs ad-din ilaha huwa q...,৪ <ar>مَـٰلِكِ يَوْمِ ٱلدِّينِ</ar> ৪ প্রতি...
